In [ ]:
!pip install transformers

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
model = AutoModelForSeq2SeqLM.from_pretrained("t5-base")
tokenizer = AutoTokenizer.from_pretrained("t5-base")

KeyboardInterrupt: ignored

In [ ]:
#Input processing
import os
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#run to increase field_size_limit of the csv file
import sys
import csv
maxInt = sys.maxsize

while True:
    # decrease the maxInt value by factor 10 
    # as long as the OverflowError occurs.

    try:
        csv.field_size_limit(maxInt)
        break
    except OverflowError:
        maxInt = int(maxInt/10)

In [ ]:
import pandas as pd
df=pd.read_csv('/content/drive/My Drive/sp500-1.csv', sep='~'  , engine='python')

In [ ]:
validation_years= [2020,2008,2012, 2016]
validation = df[df['year'].isin(validation_years)]
test_years = [2009, 2013, 2017, 2021]
test = df[df['year'].isin(test_years)]
train = df[~df['year'].isin(validation_years+test_years)]

In [ ]:
# train = train.sample(frac=1)
train_transcripts = train["transcript"][:1000]
train_volat = train["volat3"][:1000]
# test_transcripts = test["transcript"][:1000]
# test_volat = test["volat3"][:1000]

In [ ]:
import torch
import pickle

device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = model.to(device)

res = []
print("summarization training: ")
for i in range(0, len(train_transcripts), 10):
  tokens = tokenizer(train_transcripts[i:i+10].tolist(), max_length=512, truncation=True, return_tensors="pt")
  outputs = model.generate(tokens["input_ids"].to(device), max_length=150, min_length=140, length_penalty=1, early_stopping=True).to(device)
  text = tokenizer.decode(outputs[0])
  res.append(text)
  if i % 100 == 0:
    with open(f'/content/drive/My Drive/hidden{i}.pkl', "wb") as file:
        pickle.dump([res, y], file)
    print(i)


KeyboardInterrupt



In [ ]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification, BertConfig


tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
config = BertConfig.from_pretrained("ProsusAI/finbert", output_hidden_states=True)
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert", config=config)

Downloading:   0%|          | 0.00/252 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/758 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

In [ ]:
train_transcripts = train["transcript"][:1000]
train_volat = train["volat3"][:1000]

In [ ]:
import pickle
# crashes on colab pro+
ret = []
y = []

print("extracting training: ")
for i in range(0, len(train_transcripts),8):
  tokens = tokenizer(train_transcripts[i:i+8].tolist(), padding=True, max_length=512, truncation=True, return_tensors="pt")
  outputs = model(**tokens)
  y.append(train_volat[i:i+8])
  ret.append(outputs.hidden_states[-1])
  if i % 50 == 0:
    with open(f'/content/drive/My Drive/train_hidden{i}.pkl', "wb") as file:
        pickle.dump([ret, y], file)
        print("dumped ", i)

extracting training: 
dumped  0


In [ ]:
device = torch.device('cuda:0')
model = model.to(device)

In [ ]:
hidden_test = [ret, y]
with open('/content/drive/My Drive/test_final.pkl', "wb") as file:
    pickle.dump(hidden_test, file)

NameError: ignored

In [ ]:
import pickle
with open('/content/drive/My Drive/test_final.pkl', "rb") as file:
    hidden_test = pickle.load(file)

In [ ]:
x, y = hidden_test[0], hidden_test[1]

In [ ]:
len(y)

32

In [ ]:
import torch
import numpy as np
y = np.hstack(y)
x = torch.vstack(x)

def normalize_list_numpy(data):
    normalized_list = (data - data.mean()) / data.std()
    return normalized_list

scaled_y = normalize_list_numpy(y)

In [ ]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(x, scaled_y, test_size=0.2)
train_x = train_x.type(torch.FloatTensor)
train_y = torch.from_numpy(train_y)
train_y = train_y.type(torch.FloatTensor)

In [ ]:
from torch import nn
from torch.utils.data import DataLoader
train_data = []

trainloader = torch.utils.data.DataLoader([ [train_x[i], train_y[i]] for i in range(len(train_x))], shuffle=True, batch_size=8)

In [ ]:
def initialize_weights(m):
  if isinstance(m, nn.Linear):
      nn.init.kaiming_uniform_(m.weight.data)

In [ ]:
from torch.nn.functional import normalize

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

#https://stackoverflow.com/questions/62372938/understanding-input-shape-to-pytorch-conv1d
class NeuralNetwork(nn.Module):
   def __init__(self):
      super(NeuralNetwork, self).__init__()
      self.flatten = nn.Flatten(start_dim=1)
      self.loss = nn.MSELoss()
      self.linear_relu_stack = nn.Sequential(
      nn.Linear(768*512, 512),
      nn.LayerNorm(512),
      nn.LeakyReLU(),
      nn.Linear(512, 256),
      nn.LeakyReLU(.01),
      nn.Linear(256, 128),
      nn.ReLU(),
      nn.Linear(128, 1)
      )

   def forward(self, x):
      x = normalize(x, p=2.0)
      transpose = x.permute(0,2,1)
      transpose = self.flatten(transpose)
      logits = self.linear_relu_stack(transpose)
      
      return logits

model2 = NeuralNetwork()
model2.apply(initialize_weights)
# model.to(device)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (loss): MSELoss()
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=393216, out_features=512, bias=True)
    (1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    (2): LeakyReLU(negative_slope=0.01)
    (3): Linear(in_features=512, out_features=256, bias=True)
    (4): LeakyReLU(negative_slope=0.01)
    (5): Linear(in_features=256, out_features=128, bias=True)
    (6): ReLU()
    (7): Linear(in_features=128, out_features=1, bias=True)
  )
)

In [ ]:
#model params
from torch.optim import SGD
from transformers import get_scheduler
import torch.nn.functional as F

optimizer = SGD(model2.parameters(), lr=0.001, momentum=0.9)
num_epochs = 1000
num_training_steps = num_epochs * len(trainloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)


In [ ]:
model2.to(device)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (loss): MSELoss()
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=393216, out_features=512, bias=True)
    (1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    (2): LeakyReLU(negative_slope=0.01)
    (3): Linear(in_features=512, out_features=256, bias=True)
    (4): LeakyReLU(negative_slope=0.01)
    (5): Linear(in_features=256, out_features=128, bias=True)
    (6): ReLU()
    (7): Linear(in_features=128, out_features=1, bias=True)
  )
)

In [ ]:
from tqdm.auto import tqdm
from sklearn.metrics import mean_absolute_error

progress_bar = tqdm(range(num_training_steps))
criterion = nn.MSELoss()
model2.train()
losses = []

for epoch in range(num_epochs):
    for batch, y in trainloader:
        outputs = model2(batch.to(device))
        loss = criterion(outputs, y.to(device))
        loss = loss.type(torch.FloatTensor)
        loss.backward(retain_graph=True)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)
    if epoch % 50 == 0:
        outputs = outputs.cpu()
        outputs = outputs.detach().numpy()
        mae = mean_absolute_error(np.nan_to_num(outputs), y.cpu())
        print("at epoch :", epoch)
        print("mae= ", mae)
        print("loss =", loss)
        print("outputs = ", outputs)
        print("real =", y)

  0%|          | 0/4000 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([8])) that is different to the input size (torch.Size([8, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([8])) that is different to the input size (torch.Size([8, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, 

at epoch : 0
mae=  1.9577487
loss = tensor(3.8328, grad_fn=<ToCopyBackward0>)
outputs =  [[0.59131473]]
real = tensor([-1.3664])


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


at epoch : 50
mae=  1.2528343
loss = tensor(1.5696, grad_fn=<ToCopyBackward0>)
outputs =  [[-0.03581619]]
real = tensor([-1.2887])


...

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([8])) that is different to the input size (torch.Size([8, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


at epoch : 900
mae=  0.48169512
loss = tensor(0.2320, grad_fn=<ToCopyBackward0>)
outputs =  [[0.2186798]]
real = tensor([0.7004])


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([8])) that is different to the input size (torch.Size([8, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


at epoch : 950
mae=  0.6443084
loss = tensor(0.4151, grad_fn=<ToCopyBackward0>)
outputs =  [[0.38579375]]
real = tensor([1.0301])


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([8])) that is different to the input size (torch.Size([8, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [ ]:
with torch.no_grad():
    
    outputs = model2(x.to(device))
    print('outputs: ', outputs)
    print(outputs.shape, y.shape)
    print('yTest: ', y)
    loss = criterion(y, outputs)
    print('loss : ', loss)
    print(loss.shape)
    x = loss
    mae = mean_absolute_error(y.cpu(), outputs.cpu())
    print('mae: ', mae)

outputs:  tensor([[ 0.4826],
        [ 0.1366],
        [ 0.3201],
        [ 0.0925],
        [ 0.2945],
        [-0.3777],
        [ 0.2531],
        [-0.2391],
        [ 0.1236],
        [-0.0687],
        [ 0.3778],
        [-0.0859],
        [ 0.3336],
        [-0.4299],
        [ 0.2174],
        [-0.0636],
        [ 0.2287],
        [-0.3677],
        [ 0.2379],
        [-0.3597],
        [ 0.3711],
        [-0.0236],
        [ 0.4054],
        [ 0.1236],
        [ 0.1721],
        [-0.0655],
        [ 0.3119],
        [-0.3408],
        [ 0.2638],
        [ 0.1621],
        [ 0.1107],
        [-0.4941]], device='cuda:0')
torch.Size([32, 1]) (32,)
yTest:  [ 0.46557999  0.0266131   0.31367287 -0.21327181  0.26060599 -0.20547375
  0.23985706 -0.15785113  0.42579979  0.00784835  0.34401882  0.00723172
  0.32222462 -0.25453565  0.2500886   0.00552383  0.26471749 -0.2639026
  0.22156484 -0.18832888  0.32276651  0.00343661  0.33766571 -0.005499
  0.19491883  0.00844034  0.28851876 -0.1

TypeError: ignored

In [ ]:
outputs = outputs.numpy()

In [ ]:
y

array([ 0.46557999,  0.0266131 ,  0.31367287, -0.21327181,  0.26060599,
       -0.20547375,  0.23985706, -0.15785113,  0.42579979,  0.00784835,
        0.34401882,  0.00723172,  0.32222462, -0.25453565,  0.2500886 ,
        0.00552383,  0.26471749, -0.2639026 ,  0.22156484, -0.18832888,
        0.32276651,  0.00343661,  0.33766571, -0.005499  ,  0.19491883,
        0.00844034,  0.28851876, -0.16065584,  0.24386668,  0.00506019,
        0.21352769, -0.26120725])

In [ ]:
df['prediction'] = outputs

In [ ]:
from google.colab import files
df.to_csv('filename.csv') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!cp filename.csv "drive/My Drive/"